# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
import numpy as np
import re
from sqlalchemy import create_engine
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sanaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sanaa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disasterdb.db')
df= pd.read_sql_table('disaster', engine)

In [3]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [6]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001D38C20CD38>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2, random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def perf_report(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [9]:
perf_report(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.76      0.27      0.40      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.81      5206
   macro avg       0.79      0.62      0.64      5206
weighted avg       0.80      0.81      0.77      5206

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4304
           1       0.89      0.45      0.60       902

    accuracy                           0.90      5206
   macro avg       0.89      0.72      0.77      5206
weighted avg       0.89      0.90      0.88      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

C:\Users\sanaa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5105
           1       0.80      0.04      0.08       101

    accuracy                           0.98      5206
   macro avg       0.89      0.52      0.53      5206
weighted avg       0.98      0.98      0.97      5206

tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5176
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      0.99      0.99      5206

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5157
           1       0.00      0.00      0.00        49

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.98      0.99      0.99      5206

sho

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [19]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
# predict on test data
perf_report(cv, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.81      5206
   macro avg       0.79      0.63      0.65      5206
weighted avg       0.80      0.81      0.77      5206

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4304
           1       0.89      0.45      0.60       902

    accuracy                           0.90      5206
   macro avg       0.89      0.72      0.77      5206
weighted avg       0.89      0.90      0.88      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

C:\Users\sanaa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      5046
           1       0.67      0.03      0.05       160

    accuracy                           0.97      5206
   macro avg       0.82      0.51      0.52      5206
weighted avg       0.96      0.97      0.96      5206

death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4981
           1       0.89      0.15      0.25       225

    accuracy                           0.96      5206
   macro avg       0.93      0.57      0.62      5206
weighted avg       0.96      0.96      0.95      5206

other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      4539
           1       0.74      0.02      0.04       667

    accuracy                           0.87      5206
   macro avg       0.81      0.51      0.49      5206
weighted avg       0.86      0.87      0.82      5206

inf

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [32]:
#Improve  the pipeline
pipeline2  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0))))
    ])

In [33]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001D38C20CD38>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=True,
         

In [34]:
#Train & predict
pipeline2.fit(X_train, y_train)
perf_report(pipeline2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.68      0.53      0.60      1247
           1       0.86      0.92      0.89      3959

    accuracy                           0.83      5206
   macro avg       0.77      0.73      0.75      5206
weighted avg       0.82      0.83      0.82      5206

request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4304
           1       0.77      0.62      0.69       902

    accuracy                           0.90      5206
   macro avg       0.85      0.79      0.81      5206
weighted avg       0.90      0.90      0.90      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

C:\Users\sanaa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5151
           1       0.69      0.16      0.26        55

    accuracy                           0.99      5206
   macro avg       0.84      0.58      0.63      5206
weighted avg       0.99      0.99      0.99      5206

refugees
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5046
           1       0.54      0.24      0.33       160

    accuracy                           0.97      5206
   macro avg       0.76      0.62      0.66      5206
weighted avg       0.96      0.97      0.96      5206

death
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4981
           1       0.76      0.48      0.58       225

    accuracy                           0.97      5206
   macro avg       0.87      0.73      0.78      5206
weighted avg       0.97      0.97      0.97      5206

othe

In [47]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters, verbose=2, cv=3, scoring='f1_macro')

In [48]:
cv2.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 2.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 2.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, total= 8.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 114.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 9. Export your model as a pickle file

In [50]:

with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.